In [ ]:
# !pip install tensorflow-datasets
!pip install -U tfds-nightly

In [ ]:
# !pip install -U --pre tensorflow=="2.1"
# !pip install tf_slim

In [ ]:
# !pip install pycocotools

In [ ]:
# !pip install tensorflow==2.1


In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display



In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
import tensorflow_datasets as tfds


In [ ]:
!gcloud config set account application-default

In [ ]:

import pandas as pd
import numpy as np
import time

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import model_from_json
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard


# ----------- import data and scaling ----------- #
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

X_train = df_train[['xmin', 'ymin', 'xmax', 'ymax']].values
y_train = df_train[['xloc', 'yloc','zloc']].values

X_test = df_test[['xmin', 'ymin', 'xmax', 'ymax']].values
y_test = df_test[['xloc', 'yloc','zloc']].values

# standardized data
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
y_train = scalar.fit_transform(y_train)

In [ ]:
# TRAINING MODE

# ----------- create model ----------- #
model = keras.Sequential()
model.add(Dense(6, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, kernel_initializer='normal', activation='relu'))
model.add(Dense(4, kernel_initializer='normal', activation='relu'))
model.add(Dense(3, kernel_initializer='normal'))

# model = multi_gpu_model(model, gpus=2)
model.compile(loss='mean_squared_error', optimizer='adam')

# ----------- define callbacks ----------- #
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7,
                verbose=1, min_delta=1e-4, mode='min')
modelname = "model@{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir="logs/{}".format(modelname))

# ----------- start training ----------- #
history = model.fit(X_train, y_train,
            validation_split=0.1, epochs=7000, batch_size=2048, callbacks=[tensorboard])


# ----------- save model and weights ----------- #
model_json = model.to_json()
with open("generated_files/{}.json".format(modelname), "w") as json_file:
   json_file.write(model_json)

model.save_weights("generated_files/{}.h5".format(modelname))
print("Saved model to disk")


In [ ]:
# INFERENCE MODE

from tensorflow.keras.models import model_from_json

df_test = pd.read_csv('data/test.csv')
X_test = df_test[['xmin', 'ymin', 'xmax', 'ymax']].values
y_test = df_test[['xloc' ,'yloc','zloc']].values

# standardized data
scalar = StandardScaler()
X_test = scalar.fit_transform(X_test)
y_test = scalar.fit_transform(y_test)
    
MODEL = "model@1594929980"
WEIGHTS = "model@1594929980"

# load json and create model
json_file = open('generated_files/{}.json'.format(MODEL), 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json( loaded_model_json )

# load weights into new model
loaded_model.load_weights("generated_files/{}.h5".format(WEIGHTS))
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='mean_squared_error', optimizer='adam')
y_pred = loaded_model.predict(X_test)

# scale up predictions to original values
y_pred = scalar.inverse_transform(y_pred)
y_test = scalar.inverse_transform(y_test)

# save predictions
df_result = df_test
df_result['zloc_pred'] = -100000

for idx, row in df_result.iterrows():
    df_result.at[idx, 'zloc_pred'] = y_pred[idx]

df_result.to_csv('data/predictions.csv', index=False)

In [ ]:
y_pred[:10]

In [ ]:
model.predict( [X_test[0]])

In [ ]:
loss, mae, mse = model.evaluate(X_test, y_test, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))